# Readymade Data Module Assignment
## NYC 311 Service Request Analysis

**Student Name:** Nick Pisarczyk

**U-M Unique Name:** npisar

**Research Question:** State your chosen research question here

---

**BEFORE YOU START:**
1. Read the assignment instructions on Canvas carefully.
2. Make a copy of this notebook and work on your own copy.
3. Understand the dataset before you start cleaning and analyzing. NYC Open Data has a nice portal and a data dictionary for exploring their datasets.
4. NYC 311 is a **very large** dataset. When you are fetching data from the portal or API, we would recommend you to first think about your research questions and start with a small subset of the data and then increase the size of the data as you get more comfortable with the data. Generally, you do **not** need to use the entire dataset to answer your research question.
5. This notebook serves as a template. You can add more cells or make adjustments as you see fit. But make sure to keep all the sections mentioned in the assignment instructions. Also, format your notebook properly for better readability.

## Data Statement

Describe your data source here. Include:
- Where you obtained the data (URL or API endpoint)
- What subset you're analyzing (dates, geography, etc.)
- Any filters or sampling you applied
- File size/number of records

## Assignment Details (Canvas)

https://umich.instructure.com/courses/825993/assignments/3016485

**Overview**
This assignment asks you to analyze NYC 311 service request data to understand patterns in urban neighborhoods. You will use Python to explore relationships between service requests and socioeconomic characteristics, practicing skills in data cleaning, exploratory analysis, visualization, and interpretation.

311 systems allow residents to report non-emergency issues like noise complaints, street conditions, and building problems. As Wang et al. (2017) demonstrated, these service request patterns can reveal distinctive “signatures” of urban neighborhoods that correlate with demographic and economic characteristics.

---

**Learning Objectives**<br>
By completing this assignment, you will:<br>
- Practice working with “readymade” administrative data from government sources
- Develop skills in data cleaning and exploratory data analysis
- Apply statistical methods to understand relationships in observational data
- Create effective visualizations to communicate data patterns
- Consider methodological limitations and ethical implications of using administrative data for research
- Data

---

**NYC Open Data - 311 Service Requests**

You will use NYC’s publicly available 311 service request dataLinks to an external site..

Recommended approach: Explore the dataset using the NYC Open Data online portal and the data dictionary. The full dataset is very large (40+ million records), so filtering is essential. For instance, one year of data was around 2.6 GB. 

Based on your research question, identify an appropriate subset of the data (e.g., one year of data for specific boroughs or zip codes), and use the Socrata API or directly download it.You can find more information about the API and the documentation on NYC Open Data website.

---

**Research Questions**
Choose ONE of the following research questions to investigate:

- Option 1: Neighborhood Service Request Signatures
<br>Do different neighborhoods have distinctive patterns in their 311 service requests? Analyze the relative frequency of complaint types across neighborhoods. What patterns do you notice?

- Option 2: Temporal Patterns in Service Requests
<br>How do 311 service requests vary by time of day, day of week, or season? Are there particular complaint types that show strong temporal patterns? What might explain these patterns?

- Option 3: Response Time Disparities
<br>Do response times for 311 requests differ across neighborhoods or complaint types? Calculate the time between request creation and closure, and examine whether there are systematic differences by location or complaint category.

- Option 4: Complaint Type Evolution
<br>How have the types and volumes of 311 requests changed over time (comparing multiple years)? What might explain these patterns?

---

**Assignment Requirements**<br>
Your submission should include two components. Both should be included in a single Jupyter Notebook.
<br>
1. Data Analysis<br>
Your notebook should include:

- Data loading and initial exploration: Load the data, explain how you retrieve the data, why you choose the specific subset, examine its structure, and identify relevant variables
- Data cleaning: Handle missing values, filter to relevant records, create derived variables as needed
- Exploratory analysis: Use descriptive statistics and visualizations to understand patterns
- Focused analysis: Apply appropriate statistical methods to address your research question
- Visualizations: Create at least 3 meaningful visualizations (e.g., time series, bar charts, heatmaps, scatter plots)
- Documentation: Include markdown cells explaining your approach, interpreting results, and noting limitations
- Technical expectations: Use numpy/pandas for data manipulation, matplotlib/seaborn for visualization, and appropriate statistical libraries (scipy, statsmodels, etc.) as needed.

2. Written Summary<br>
Briefly report on the following. Your notebook should have a section for each bullet point.

- Research question and motivation: What is your research question? Why is this question interesting? What might we learn?
- Methods: Describe your data source, cleaning steps, and analytical approach
- Findings: Summarize key patterns and statistical results (include 2-3 key visualizations)
- Limitations: What are the methodological limitations? What can and cannot be concluded from this analysis?
- Ethical considerations: Reflect on the use of 311 data for research. Who is represented? Who might be excluded? What are potential concerns about using this data?
 
---

**Assignment Submission**

Please upload your Jupyter Notebook file to Canvas by the deadline: A single .ipynb file that includes all your work. 

Before submitting the file, ensure that your notebook is properly running on your machine. Ensure your notebook contains all your responses, plots, all programming parts of this assignment and is properly formatted with headings, explanations and code comments.

If you use Generative AI tools in any part of your assignment, you need to follow the AI policy in our syllabus and document your AI use at the end of your notebook.

## 1. Setup and Data Loading

In [1]:
# Import libraries
# You can import any libraries you may need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sodapy import Socrata

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [11]:
# save to csv helper function
def df_to_csv(d):
    for borough, df in d.items():
        df.to_csv(f'data/readymade_module_data/{borough}_data.csv', index=False)

### Option A: Load from downloaded CSV file

**Note**: If you are running the notebook using `colab` kernel, you **cannot** directly import the data from your own laptop. Please see the class repo README files for more details

In [3]:
# Load your downloaded data
# df = pd.read_csv('311_Service_Requests.csv', 
#                  parse_dates='Created Date', 'Closed Date',
#                  low_memory=False)

### Option B: Load from Socrata API (recommended for smaller datasets)

In [ ]:
# Example: Get 100,000 records from 2023 for Brooklyn
client = Socrata("data.cityofnewyork.us", None)

# specify columns to reduce runtimes and get only pertinent data
columns = [
    "unique_key",
    "created_date",
    "closed_date",
    "complaint_type",
    "descriptor",
    "descriptor_2",
    "borough"
]
select_str = ', '.join(columns)

# define the 5 NYC boroughs (neighborhoods)
boroughs = [
    "BRONX",
    "BROOKLYN",
    "MANHATTAN",
    "QUEENS",
    "STATEN ISLAND"
]

dfs = {}

# find data for each borough and create 5 separate dfs
for borough in boroughs:
    print(f"Gathering data for borough: {borough}...")
    results = client.get("erm2-nwe9", 
                        select=select_str,
                        where=f"created_date >= '2026-01-01' AND borough = {borough}",
                        limit=100000)
    dfs[borough.lower()] = pd.DataFrame.from_records(results)

    print(f"Converting time columns...")
    # Convert date columns
    dfs[borough.lower()]['created_date'] = pd.to_datetime(dfs[borough.lower()]['created_date'])
    dfs[borough.lower()]['closed_date'] = pd.to_datetime(dfs[borough.lower()]['closed_date'])

    if not borough == "STATEN ISLAND":
        print(f"Complete! Next bourough...\n\n")
    else:
        print(f"Complete for all boroughs!")
    
print(f"dfs dict is:\n{dfs}")


Gathering data for borough: BRONX...


HTTPError: 400 Client Error: Bad Request.
	Query coordinator error: query.soql.no-such-column; No such column: BRONX; position: Map(row -> 1, column -> 167, line -> "SELECT `unique_key`, `created_date`, `closed_date`, `complaint_type`, `descriptor`, `descriptor_2`, `borough` WHERE (`created_date` >= \"2026-01-01\") AND (`borough` = `BRONX`) LIMIT 100000\n                                                                                                                                                                      ^")

In [ ]:
# save to CSV for easier access
df_to_csv(dfs)

## 2.Data Description

You can describe the data in many ways. Here are some baseline requirements:
- Display basic information about the dataset (what are the relevant variables? What are their types? How many observations are there?)
- Conduct summary statistics of the relevant variables
- Check for missing values

In [8]:
# You can have as many cells as you want
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   unique_key      100000 non-null  str           
 1   created_date    100000 non-null  datetime64[us]
 2   closed_date     91957 non-null   datetime64[us]
 3   complaint_type  100000 non-null  str           
 4   descriptor      100000 non-null  str           
 5   city            96605 non-null   str           
 6   borough         100000 non-null  str           
 7   descriptor_2    52233 non-null   str           
dtypes: datetime64[us](2), str(6)
memory usage: 6.1 MB


## 3. Data Cleaning

Document your cleaning decisions and rationale here

In [6]:
# Example cleaning steps (customize based on your needs)

# 1. Remove rows with missing essential data
# df_clean = df.dropna(subset='created_date', 'complaint_type')

# 2. Filter to specific time period if needed
# df_clean = df_clean(df_clean'created_date' >= '2023-01-01') & 
#                     (df_clean'created_date' < '2024-01-01')

# 3. Create derived variables
# Example: Calculate response time
# df_clean'response_time_hours' = (
#     (df_clean'closed_date' - df_clean'created_date').dt.total_seconds() / 3600
# )

# Example: Extract temporal features
# df_clean'hour' = df_clean'created_date'.dt.hour
# df_clean'day_of_week' = df_clean'created_date'.dt.dayofweek
# df_clean'month' = df_clean'created_date'.dt.month

print(f"Original dataset: {len(df)} records")
# print(f"Cleaned dataset: {len(df_clean)} records")
# print(f"Removed: {len(df) - len(df_clean)} records ({((len(df) - len(df_clean))/len(df)*100):.1f}%)")

NameError: name 'df' is not defined

## 4.Exploratory Data Analysis

Add narrative about what you're exploring, why and what you've found

In [ ]:
# Example: Most common complaint types
# complaint_counts = df_clean'complaint_type'.value_counts().head(15)
# plt.figure(figsize=(12, 6))
# complaint_counts.plot(kind='barh')
# plt.xlabel('Number of Requests')
# plt.ylabel('Complaint Type')
# plt.title('Top 15 Most Common 311 Complaint Types')
# plt.tight_layout()
# plt.show()

In [ ]:
# Example: Temporal patterns
# requests_by_month = df_clean.groupby('month').size()
# plt.figure(figsize=(12, 6))
# requests_by_month.plot(kind='bar')
# plt.xlabel('Month')
# plt.ylabel('Number of Requests')
# plt.title('311 Requests by Month')
# plt.xticks(rotation=0)
# plt.tight_layout()
# plt.show()

## 5.Research Question Analysis

This is the core of your assignment. Document your analytical approach here. You can add any cells if you see fit.

In [ ]:
# Your focused analysis goes here
# This will vary significantly based on your research question. You can organize your analysis as you like. You can have as many cells as you want.

### Statistical Testing (if applicable)

In [ ]:
# Example: Statistical tests
# from scipy import stats

# # t-test example
# group1 = df_cleandf_clean'borough' == 'MANHATTAN''response_time_hours'.dropna()
# group2 = df_cleandf_clean'borough' == 'BRONX''response_time_hours'.dropna()
# t_stat, p_value = stats.ttest_ind(group1, group2)
# print(f"t-statistic: {t_stat:.3f}")
# print(f"p-value: {p_value:.3f}")

## Key Visualizations

Create at least 3 polished visualizations that answer your research question

In [ ]:
# Visualization 1
# Your code here

Interpretation of Visualization 1

In [ ]:
# Visualization 2
# Your code here

Interpretation of Visualization 2

In [ ]:
# Visualization 3
# Your code here

Interpretation of Visualization 3

## Written Summary

Summarize your key findings here. What patterns did you discover? What can you conclude?

### Research Question and Motivation

- Why is this question interesting? 
- What might we learn?

### Methods

Describe your data source, cleaning steps, and analytical approach

### Findings

Summarize key patterns and statistical results (refer to your key visualizations)

### Limitations

Discuss methodological limitations. What are the potential biases in 311 data? What alternative explanations exist for your findings? What can and cannot be concluded from this analysis?

### Ethical Considerations

Reflect on the ethical implications of using 311 data:
- Who is represented in this data? Who might be underrepresented?
- What are potential privacy concerns?
- How might this analysis be used or misused?
- What are the implications for equity and justice?

## AI Appendix (if applicable)

If you used AI during this assignnment, explain
1. what part of the work it was used for; 
2. what AI tools you used; 
3. the prompts you used; 
4. how you analyzed the AI work for accuracy; and, 
5. steps you took to rework and revise your final documents so that they were both factually accurate and reflected your own voice and style.


## Submission

Submit your assignment as .ipynb file. Make sure to double check with the assignment instructions on Canvas.